In [203]:
from os import times
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from setuptools.command.install import install
from selenium.webdriver.support import expected_conditions as EC
from trio import sleep
import requests
import base64


In [204]:
def openDriver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    DRIVER_PATH = "chromedriver.exe"
    return webdriver.Chrome(options=chrome_options, service=Service(DRIVER_PATH))


In [205]:
def getLink(uri):
    driver = openDriver()
    driver.get(uri)
    time.sleep(1)
    
    links = []
    parent = driver.find_elements(By.CLASS_NAME, "plp-fragment-wrapper")
    for index in parent:
        root = ((index.find_element(By.CLASS_NAME, "plp-mastercard ").
                 find_element(By.TAG_NAME, "a")).
                get_attribute("href"))
        links.append(root)
        
    driver.quit()
    return links


In [206]:
import json
import os

def saveDataToJSON(directory, filename, data):
    os.makedirs("json/link/" + directory, exist_ok=True)
    try:
        with open(f'json/link/{directory}/{filename}.json', 'r', encoding='utf-8') as FILE:
            existingData = json.load(FILE)
            
    except FileNotFoundError:
        existingData = []
    
    existingData.extend(data)
    with open(f'json/link/{directory}/{filename}.json', 'w', encoding='utf-8') as FILE:
        json.dump(existingData, FILE, ensure_ascii=False, indent=4)
        

In [ ]:
# for i in range(12,14):
#     url = {
#         "furniture sets":
#                f"https://www.ikea.com/th/th/cat/furniture-sets-55036/?page={i}"
#     }
#     LinkList = getLink(url.get('furniture sets'))
#     saveDataToJSON("furniture sets","FurnitureSetsLink",LinkList)
#     print("page = ",i)
#     

In [207]:
def getLinkProduct(n, nameCat, urlCat):
    for i in range(1, n):
        uri = {
            nameCat:
                f"{urlCat}?page={i}"
        }
        linkList = getLink(uri.get(nameCat))
        saveDataToJSON(nameCat, f"{nameCat}Link", linkList)
        print("page = ", i)

In [208]:
def getDetailProduct(urlCat):
    driver = openDriver()
    driver.get(urlCat)
    
    time.sleep(1)
    allDetail = driver.find_element(By.XPATH, '//div[@class="js-price-package pip-price-package"]')
    brandName = allDetail.text.split("\n")[0]
    detail = allDetail.text.split("\n")[1].split(", ")
    productName = detail[0]
    try:
        color = detail[1]
    except:
        color = None
    try:
        size = detail[2]
    except:
        size = None
    price = allDetail.text.split("\n")[2][:-3]
    
    driver.quit()
    return brandName, productName,color, size, price

In [ ]:
brandName, productName,color, size, price = getDetailProduct("https://www.ikea.com/th/th/p/kallax-shelving-unit-white-20351884/")
print("BrandName :",brandName)
print("ProductName :",productName)
print("Color:",color)
print("Size:",size)
print("Price:",price)


-----------------------------------------------------------------------------------------

###กดปุ่ม เพื่อรับข้อมูล

In [217]:
def getDescFromProduct(uri):
    browser = openDriver()
    browser.get(uri)
    try:
        button = browser.find_element(By.CLASS_NAME, "pip-list-view-item__action")
        browser.execute_script("arguments[0].click();", button)
        time.sleep(1)
        paragraphs = browser.find_elements(By.CLASS_NAME, 'pip-product-details__paragraph')
        codeProduct = (browser.find_element(By.CLASS_NAME, 'pip-product-identifier__value').
                       get_attribute("innerText"))
        
        stringDesc = ""
        for paragraph in paragraphs:
            stringDesc += paragraph.get_attribute("innerText")+"\n\n"

        browser.quit()
        return stringDesc , codeProduct
        
    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")
        browser.quit()

------------------------

In [210]:
# Download Image for Product
def saveImage(uri, dirName, fileName):
    response = requests.get(uri)
    
    if response.status_code == 200:
        os.makedirs("image/"+dirName, exist_ok=True)
        with open(f"image/{dirName}/{fileName}_image.jpg", "wb") as file:
            file.write(response.content)
    else:
        print("เกิดข้อผิดพลาดในการดาวน์โหลดภาพ:", response.status_code)


In [211]:
def kernelImage(dirName, fileName):
    with open(f"image/{dirName}/{fileName}_image.jpg", "rb") as png_file:
        # อ่านไฟล์ PNG
        png_data = png_file.read()
        # แปลงข้อมูลไฟล์ PNG เป็น Base64
        base64_encoded_data = base64.b64encode(png_data)
        # แปลง Base64 เป็น string เพื่อใช้งาน
        base64_string = base64_encoded_data.decode('utf-8')
        
    return base64_string
    
    

In [212]:
def getImage(uri, dirName, fileName):
    driver = openDriver()
    driver.get(uri)
    findElement = driver.find_element(By.XPATH, "//div[@class='pip-zoom-image']")
    img_element = findElement.find_element(By.TAG_NAME, "img")
    srcset = img_element.get_attribute("srcset")
    
    first_link = srcset.split(",")[0].strip().split(" ")[0]
    
    driver.quit()
    saveImage(first_link, dirName, fileName)
    return kernelImage(dirName, fileName)
    

In [213]:
import pandas as pd

In [214]:
def readJson(pathname, filename):
    data = pd.read_json("json/link/"+pathname+"/"+filename)
    return data

In [215]:
def saveProductDataToJson(dirName, fileName):
    links = readJson(dirName, fileName)
    datas = []
    try:
        for i in links[0]:
            brandName, productName,color, size, price = getDetailProduct(i)
            stringDesc , codeProduct = getDescFromProduct(i)
            imgBase64 = getImage(i, dirName, codeProduct)
    
            datas.append({
                "Code":codeProduct,
                "brandName":brandName,
                "productName":productName,
                "color":color,
                "size":size,
                "price":price,
                "imgBase64":imgBase64,
                "desc":stringDesc
            })
            
        datas = {dirName:datas}
        os.makedirs("json/product/" + dirName, exist_ok=True)
        try:
            with open(f'json/product/{dirName}/{dirName}_datas.json', 'w', encoding='utf-8') as FILE:
                json.dump(datas, FILE, ensure_ascii=False, indent=4)
        except FileNotFoundError:
            print(" Can not save data")
            
    except Exception as e:
        print(e)

In [199]:
# n = 0
# datas = []

In [ ]:
saveProductDataToJson("baskets", "basketsLink.json")

In [ ]:
d = {
    "bed":{
    {
    "ID":"1233",
    "name":"hello",
    "price":1,
    "size":1,
    "desc":"Hi",
    "color":"blue",
    "stock":1,
    "img":"base64"
    },
    {
    "name":"hello",
    "price":1,
    "size":1,
    "desc":"Hi",
    "color":"blue",
    "stock":1,
    "img":"base64"
    }
    }
}